# Filesystem e accesso ai servizi dell'OS
  
Da [File and Directory Access](https://docs.python.org/3/library/filesys.html),
  [Generic Operating System Services](https://docs.python.org/3/library/allos.html) e
  [Subprocess management](https://docs.python.org/3/library/subprocess.html).
 
## Path orientati ad oggetti

In [195]:
from pathlib import Path

In [120]:
# relativo e assoluto

(
    Path(), 
    Path().absolute(), 
    Path('/Users/santini/Documents/Activities').relative_to('/Users')
)

(PosixPath('.'),
 PosixPath('/Users/santini/Documents/Activities/Teaching/Courses/pybg/repo/moduli/sysadm'),
 PosixPath('santini/Documents/Activities'))

In [121]:
# overloading di / 

Path('~santini') / 'Documents' / 'Dropbox'

PosixPath('~santini/Documents/Dropbox')

In [122]:
# globbing (ache ricorsivo)

list(Path('/etc').glob('sys*'))

[PosixPath('/etc/syslog.conf'), PosixPath('/etc/syslog.conf~previous')]

In [123]:
list(Path().rglob('*.ipynb'))

[PosixPath('01-StandardLibrary.ipynb'),
 PosixPath('02-Fabric.ipynb'),
 PosixPath('.ipynb_checkpoints/Fabric-checkpoint.ipynb'),
 PosixPath('.ipynb_checkpoints/01-StandardLibrary-checkpoint.ipynb')]

In [124]:
# listing 

list(Path().iterdir())

[PosixPath('docker'),
 PosixPath('.nodel'),
 PosixPath('01-StandardLibrary.ipynb'),
 PosixPath('.ipynb_checkpoints'),
 PosixPath('02-Fabric.ipynb')]

In [137]:
# home e cwd (sono metodi statici)

Path.home(), Path.cwd() 

(PosixPath('/Users/santini'),
 PosixPath('/Users/santini/Documents/Activities/Teaching/Courses/pybg/repo/moduli/sysadm'))

In [135]:
# espansione tilde

Path('~admin').expanduser()

PosixPath('/Users/admin')

In [126]:
# query

(
    Path('/pippo').exists(),
    Path('/Users').is_dir(),
    Path('/Users/santini/.bashrc').is_file(),
    Path('/Users/santini/.bash_profile').is_symlink(),
)

(False, True, True, True)

In [127]:
# parti del nome

p = Path('/dir/file.tar.gz')

p.name, p.stem, p.suffixes, p.suffix

('file.tar.gz', 'file.tar', ['.tar', '.gz'], '.gz')

In [128]:
# rimpiazzare nome e suffisso 

p.with_name('newname.ext'), p.with_suffix('.newsuff')

(PosixPath('/dir/newname.ext'), PosixPath('/dir/file.tar.newsuff'))

In [129]:
# parti

p = Path('/Users/santini/.bash_profile')

p.parts

('/', 'Users', 'santini', '.bash_profile')

In [130]:
# parti della directory 

p.parent, list(p.parents) 

(PosixPath('/Users/santini'),
 [PosixPath('/Users/santini'), PosixPath('/Users'), PosixPath('/')])

In [196]:
# permessi

p = Path('~/.bashrc').expanduser()

p.owner(), p.group(), p.stat()

('santini',
 'staff',
 os.stat_result(st_mode=33216, st_ino=41664083, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=3374, st_atime=1571344159, st_mtime=1557902501, st_ctime=1572017256))

In [197]:
# modificare i permessi 

old_mode = p.stat().st_mode

p.chmod(0o000)
print(oct(p.stat().st_mode))

p.chmod(old_mode)
print(oct(p.stat().st_mode))

0o100000
0o100700


In [168]:
# tutta una serie di manipolazioni

# p.rename, p.replace, p.mkdir, p.link_to…

In [198]:
# scrivere e leggere

p = Path('/tmp/test.txt')
p.write_text("""
Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.
""")

print(p.read_text().strip())

Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.


## FIle e directory temporanee

In [199]:
from tempfile import TemporaryFile, TemporaryDirectory, NamedTemporaryFile

In [200]:
with TemporaryFile() as tmp:
    tmp.write(b'123')
    tmp.seek(0)
    print(tmp.read())

b'123'


In [201]:
with TemporaryDirectory() as tmp:
    (Path(tmp) / 'file.txt').write_text('ciao')

Path(tmp).is_dir()

False

In [202]:
with NamedTemporaryFile() as tmp:
    print(tmp.name)

/var/folders/23/yqhr033n6rl4q23_2fry24j80000gn/T/tmp2ugdyb3n


## Operazioni ad alto livello su gruppi di file

In [203]:
import shutil

In [213]:
shutil.copytree('docker', 'docker-copy')

list(Path('docker-copy').iterdir())

[PosixPath('docker-copy/build_image.sh'),
 PosixPath('docker-copy/Dockerfile'),
 PosixPath('docker-copy/run_slaves.sh'),
 PosixPath('docker-copy/controller_rsa.pub'),
 PosixPath('docker-copy/run_controller.sh')]

In [214]:
shutil.rmtree('docker-copy')

Path('docker-copy').is_dir()

False

In [217]:
# il comodo which 

shutil.which('python')

'/Users/santini/.local/share/dir-venv/ee639860a61f45b0491eb86d7b9bb81c-moduli/bin/python'

## Eseguire altri programmi

In [218]:
import subprocess

In [220]:
subprocess.run(['ls', '-l'])

CompletedProcess(args=['ls', '-l'], returncode=0)

In [238]:
# catturare l'output (come testo)

result = subprocess.run(['ls', '-l'], capture_output = True, text = True)
print(result.stdout)

total 40
-rw-r--r-- 1 santini staff 14634 Oct 25 20:49 01-StandardLibrary.ipynb
-rw-r--r-- 1 santini staff 23736 Oct 25 16:43 02-Fabric.ipynb
drwxr-xr-x 6 santini staff   192 Oct 25 20:42 docker



In [239]:
# fare il parsing del comando come fa la shell

print(subprocess.run('ls -l', shell = True, capture_output = True, text = True).stdout)

total 40
-rw-r--r-- 1 santini staff 14634 Oct 25 20:49 01-StandardLibrary.ipynb
-rw-r--r-- 1 santini staff 23736 Oct 25 16:43 02-Fabric.ipynb
drwxr-xr-x 6 santini staff   192 Oct 25 20:42 docker



In [240]:
angiolieri = """S'i fosse fuoco, arderei 'l mondo;
s'i fosse vento, lo tempestarei;
s'i fosse acqua, i' l'annegherei;
""".replace("'", ' ')

subprocess.run('wc -lw', shell = True, input = angiolieri, text = True, capture_output = True).stdout

'      3      20\n'

# Aspetti di rete

Da [Networking and Interprocess Communication](https://docs.python.org/3/library/ipc.html), [Internet Data Handling](https://docs.python.org/3/library/netdata.html), 
 e [Internet Protocols and Support](https://docs.python.org/3/library/internet.html).

    
# Gestione del software Python

Da [Software Packaging and Distribution](https://docs.python.org/3/library/distribution.html).